# LangChain Framework

This notebook covers the LangChain framework - a powerful library for building applications with Large Language Models:
- **What is LangChain?**: A framework for developing LLM-powered applications
- **Core Components**: Models, Prompts, Chains, Agents, and Memory
- **Document Processing**: Loaders, text splitters, and vector stores
- **Chains**: Combining LLMs with other components
- **Agents**: Building autonomous systems that use tools
- **Memory**: Managing conversation history and context
- **Practical Applications**: Building real-world LLM applications

## Learning Objectives

- Understand what LangChain is and why it's useful
- Learn the core components of LangChain
- Work with document loaders and text splitters
- Build chains to combine LLM calls
- Create agents that can use tools
- Implement memory for conversational applications
- Build end-to-end LangChain applications


## Installation

Run this cell to install required packages (uncomment if needed):


In [ ]:
# Install packages (uncomment if needed)
# !pip install langchain langchain-openai langchain-community langchain-core chromadb sentence-transformers pypdf python-dotenv


## 1. What is LangChain?

**LangChain** is an open-source framework for building applications powered by Large Language Models (LLMs). It provides:

### Key Features:

1. **Modularity**: Break down complex LLM applications into reusable components
2. **Chains**: Combine multiple components (LLMs, prompts, tools) into sequences
3. **Agents**: Build autonomous systems that can use tools and make decisions
4. **Memory**: Manage conversation history and context
5. **Data Integration**: Connect to various data sources (documents, databases, APIs)
6. **Vector Stores**: Integrate with vector databases for RAG applications

### Why LangChain?

- **Abstraction**: Simplifies working with different LLM providers (OpenAI, Anthropic, etc.)
- **Composability**: Build complex applications from simple building blocks
- **Production-Ready**: Includes features like streaming, callbacks, and observability
- **Ecosystem**: Large community and extensive integrations

### LangChain Architecture:

```
┌─────────────┐
│   Models    │  ← LLMs, Chat Models, Embeddings
└─────────────┘
       ↓
┌─────────────┐
│   Prompts   │  ← Prompt templates, output parsers
└─────────────┘
       ↓
┌─────────────┐
│   Chains    │  ← Combine models, prompts, tools
└─────────────┘
       ↓
┌─────────────┐
│   Agents    │  ← Autonomous systems with tools
└─────────────┘
       ↓
┌─────────────┐
│   Memory    │  ← Conversation history
└─────────────┘
```


## 2. Core Components Overview

LangChain consists of several core components:

1. **Models**: Interface with different LLM providers
2. **Prompts**: Template and manage prompts
3. **Chains**: Combine multiple components
4. **Agents**: Autonomous systems that use tools
5. **Memory**: Store and retrieve conversation history
6. **Document Loaders**: Load data from various sources
7. **Text Splitters**: Split documents into chunks
8. **Vector Stores**: Store and retrieve embeddings


In [1]:
# Import core LangChain libraries
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Core LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.agents import initialize_agent, AgentType, create_react_agent
from langchain.tools import Tool

# Document processing
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Vector stores
from langchain_community.vectorstores import Chroma

print("✅ LangChain libraries imported successfully!")


✅ LangChain libraries imported successfully!


## 3. Models

LangChain provides a unified interface for working with different LLM providers. The main model types are:

- **LLMs**: Text-in, text-out models (e.g., GPT-3)
- **Chat Models**: Message-in, message-out models (e.g., GPT-4, Claude)
- **Embeddings**: Convert text to vectors

### 3.1 Chat Models (Recommended)

Chat models use a message-based interface with system, human, and AI messages.


In [ ]:
# Initialize a Chat Model (OpenAI)
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    # Create a ChatOpenAI instance
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.7,  # Controls randomness (0.0 = deterministic, 1.0 = creative)
        openai_api_key=api_key
    )
    
    # Simple invocation
    response = llm.invoke("What is machine learning in one sentence?")
    print("Response:", response.content)
else:
    print("⚠️  OPENAI_API_KEY not found in environment variables.")
    print("Please set your OpenAI API key to use the LLM.")
    print("You can use: export OPENAI_API_KEY='your-key-here'")
    llm = None


### 3.2 Using Messages

Chat models work with message objects for structured conversations.


In [ ]:
if llm:
    from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
    
    # Create messages
    messages = [
        SystemMessage(content="You are a helpful AI assistant that explains concepts simply."),
        HumanMessage(content="Explain what a neural network is in 2 sentences.")
    ]
    
    # Invoke with messages
    response = llm.invoke(messages)
    print("Response:", response.content)


## 4. Prompts

Prompts are templates that help structure inputs to LLMs. LangChain provides:

- **Prompt Templates**: Reusable prompt structures with variables
- **Chat Prompt Templates**: For chat models with system/human/AI messages
- **Output Parsers**: Structure and validate LLM outputs

### 4.1 Prompt Templates


In [ ]:
# Create a prompt template
prompt = PromptTemplate(
    input_variables=["topic", "audience"],
    template="Explain {topic} to a {audience} in simple terms. Keep it under 100 words."
)

# Format the prompt
formatted_prompt = prompt.format(topic="quantum computing", audience="10-year-old")
print("Formatted Prompt:")
print(formatted_prompt)
print("\n" + "="*50 + "\n")

# Use with LLM
if llm:
    # Note: ChatOpenAI expects messages, so we'll use ChatPromptTemplate instead
    # But PromptTemplate works with text-in/text-out LLMs
    print("Note: For ChatOpenAI, use ChatPromptTemplate (shown next)")


### 4.2 Chat Prompt Templates

Chat prompt templates are designed for chat models.


In [ ]:
# Create a chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI tutor that explains {subject} concepts."),
    ("human", "Explain {topic} in simple terms with an example.")
])

# Format with variables
messages = chat_prompt.format_messages(
    subject="computer science",
    topic="recursion"
)

print("Formatted Messages:")
for msg in messages:
    print(f"{msg.__class__.__name__}: {msg.content}")

print("\n" + "="*50 + "\n")

# Use with LLM
if llm:
    response = llm.invoke(messages)
    print("LLM Response:")
    print(response.content)


### 4.3 Output Parsers

Output parsers help structure and validate LLM responses.


In [ ]:
# Simple string output parser
output_parser = StrOutputParser()

if llm:
    # Create a chain: prompt -> LLM -> parser
    chain = chat_prompt | llm | output_parser
    
    # Invoke the chain
    result = chain.invoke({
        "subject": "mathematics",
        "topic": "Pythagorean theorem"
    })
    
    print("Parsed Output:")
    print(result)


## 5. Chains

**Chains** are sequences of calls to LLMs and other utilities. They allow you to combine multiple components:

- **LLMChain**: Basic chain with a prompt and LLM
- **Sequential Chains**: Chain multiple LLM calls together
- **Router Chains**: Route inputs to different chains
- **LCEL (LangChain Expression Language)**: Modern way to compose chains using `|` operator

### 5.1 LLMChain

The simplest chain that combines a prompt template with an LLM.


In [ ]:
if llm:
    # Create a prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        ("human", "Write a {style} summary of: {text}")
    ])
    
    # Create an LLMChain (using LCEL syntax)
    chain = prompt | llm | output_parser
    
    # Run the chain
    result = chain.invoke({
        "style": "concise",
        "text": "Machine learning is a subset of artificial intelligence that enables systems to learn from data."
    })
    
    print("Chain Result:")
    print(result)


### 5.2 Sequential Chains

Sequential chains allow you to chain multiple LLM calls where the output of one becomes the input of the next.


In [ ]:
if llm:
    # Step 1: Generate a story
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative writer."),
        ("human", "Write a short 2-sentence story about {topic}")
    ])
    story_chain = story_prompt | llm | output_parser
    
    # Step 2: Translate the story
    translate_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a translator."),
        ("human", "Translate the following text to {language}:\n\n{text}")
    ])
    translate_chain = translate_prompt | llm | output_parser
    
    # Run sequentially
    topic = "a robot learning to paint"
    story = story_chain.invoke({"topic": topic})
    print("Original Story:")
    print(story)
    print("\n" + "="*50 + "\n")
    
    translation = translate_chain.invoke({"language": "Spanish", "text": story})
    print("Translated Story:")
    print(translation)


### 5.3 LCEL (LangChain Expression Language)

LCEL is the modern way to compose chains using the `|` operator. It's more flexible and supports streaming, parallelization, and more.


In [ ]:
if llm:
    # Create a complex chain using LCEL
    prompt1 = ChatPromptTemplate.from_messages([
        ("human", "List 3 key points about {topic}")
    ])
    
    prompt2 = ChatPromptTemplate.from_messages([
        ("system", "You are a teacher explaining concepts to students."),
        ("human", "Explain these points in detail:\n{points}")
    ])
    
    # Compose chains
    chain = (
        prompt1 | llm | output_parser
    ).pipe(
        lambda x: {"points": x}
    ).pipe(
        prompt2 | llm | output_parser
    )
    
    result = chain.invoke({"topic": "neural networks"})
    print("Chain Result:")
    print(result)


## 6. Document Loaders

Document loaders help you load data from various sources (files, web pages, databases, etc.) into LangChain Document objects.

### 6.1 Text File Loader


In [ ]:
# Create a sample text file for demonstration
sample_text = """
Machine Learning Fundamentals

Machine learning is a subset of artificial intelligence that focuses on the development of algorithms 
and statistical models that enable computer systems to improve their performance on a specific task 
through experience, without being explicitly programmed.

Key Concepts:
1. Supervised Learning: Learning from labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through interaction and rewards

Applications include image recognition, natural language processing, recommendation systems, and more.
"""

# Write to a temporary file
with open("sample_ml.txt", "w") as f:
    f.write(sample_text)

# Load the document
try:
    loader = TextLoader("sample_ml.txt")
    documents = loader.load()
    
    print(f"Loaded {len(documents)} document(s)")
    print(f"Document content (first 200 chars): {documents[0].page_content[:200]}...")
    print(f"\nMetadata: {documents[0].metadata}")
except Exception as e:
    print(f"Error loading document: {e}")


### 6.2 PDF Loader

PDF loaders can extract text from PDF files. (Note: Requires pypdf package)


In [ ]:
# Example of PDF loading (commented out - requires a PDF file)
# loader = PyPDFLoader("path/to/document.pdf")
# documents = loader.load()

print("PDF loading example:")
print("To load a PDF, use: PyPDFLoader('path/to/file.pdf')")
print("This requires the pypdf package: pip install pypdf")


## 7. Text Splitters

Text splitters break documents into smaller chunks, which is essential for:
- **Vector Storage**: Embeddings work better with smaller chunks
- **Context Windows**: LLMs have token limits
- **Retrieval**: Smaller chunks improve retrieval precision

### 7.1 Recursive Character Text Splitter

The most commonly used splitter that tries to split on different separators recursively.


In [ ]:
# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,      # Maximum size of chunks (in characters)
    chunk_overlap=20,    # Overlap between chunks to preserve context
    length_function=len, # Function to measure length
)

# Split the document we loaded earlier
if 'documents' in locals() and documents:
    chunks = text_splitter.split_documents(documents)
    
    print(f"Split into {len(chunks)} chunks")
    print(f"\nChunk sizes: {[len(chunk.page_content) for chunk in chunks]}")
    print(f"\nFirst chunk:\n{chunks[0].page_content}")
    print(f"\nSecond chunk:\n{chunks[1].page_content}")
else:
    # Use sample text
    sample_doc = sample_text
    chunks = text_splitter.split_text(sample_doc)
    print(f"Split into {len(chunks)} chunks")
    for i, chunk in enumerate(chunks[:3], 1):
        print(f"\nChunk {i} ({len(chunk)} chars):\n{chunk[:100]}...")


## 8. Vector Stores and RAG

Vector stores allow you to store and retrieve document embeddings. This is the foundation of RAG (Retrieval Augmented Generation).

### 8.1 Creating a Vector Store with Chroma


In [ ]:
# Create embeddings and vector store
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    # Initialize embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    
    # Create sample documents
    sample_docs = [
        "Machine learning is a subset of AI that learns from data.",
        "Deep learning uses neural networks with multiple layers.",
        "Natural language processing helps computers understand human language.",
        "Computer vision enables machines to interpret visual information.",
        "Reinforcement learning learns through interaction and rewards."
    ]
    
    # Create vector store
    vectorstore = Chroma.from_texts(
        texts=sample_docs,
        embedding=embeddings,
        persist_directory="./langchain_chroma_db"
    )
    
    print("✅ Vector store created successfully!")
    print(f"Stored {len(sample_docs)} documents")
else:
    print("⚠️  OPENAI_API_KEY not found. Skipping vector store creation.")
    vectorstore = None


### 8.2 Retrieval and RAG Chain

Combine vector store retrieval with LLM generation.


In [ ]:
if vectorstore and llm:
    from langchain.chains import RetrievalQA
    
    # Create a retrieval chain
    retrieval_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # "stuff", "map_reduce", "refine", "map_rerank"
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True
    )
    
    # Query the chain
    query = "What is machine learning?"
    result = retrieval_chain.invoke({"query": query})
    
    print(f"Query: {query}")
    print(f"\nAnswer: {result['result']}")
    print(f"\nSource documents used: {len(result['source_documents'])}")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"\nSource {i}: {doc.page_content[:100]}...")
else:
    print("⚠️  Vector store or LLM not available. Skipping RAG example.")


### 8.3 Using LCEL for RAG

Modern way to build RAG chains using LCEL.


In [ ]:
if vectorstore and llm:
    from langchain_core.runnables import RunnablePassthrough
    
    # Create a retriever
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    
    # Create prompt template
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Use the following context to answer questions.\n\nContext: {context}"),
        ("human", "{question}")
    ])
    
    # Format documents function
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    # Create RAG chain using LCEL
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | output_parser
    )
    
    # Query
    result = rag_chain.invoke("What is deep learning?")
    print("RAG Chain Result:")
    print(result)
else:
    print("⚠️  Vector store or LLM not available. Skipping LCEL RAG example.")


## 9. Agents

**Agents** are autonomous systems that can use tools, make decisions, and take actions. They use an LLM to decide which actions to take.

### Key Concepts:

- **Tools**: Functions that agents can call (e.g., search, calculator, API calls)
- **Agent Types**: Different reasoning strategies (ReAct, Plan-and-Execute, etc.)
- **Agent Executor**: Runs the agent loop

### 9.1 Creating Simple Tools


In [ ]:
# Define custom tools
def calculate(expression: str) -> str:
    """Evaluates a mathematical expression safely."""
    try:
        # Simple calculator - in production, use a safer evaluation method
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

def get_word_length(word: str) -> str:
    """Returns the length of a word."""
    return str(len(word))

# Create Tool objects
calculator_tool = Tool(
    name="Calculator",
    func=calculate,
    description="Useful for performing mathematical calculations. Input should be a valid Python expression."
)

word_length_tool = Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word. Input should be a single word."
)

tools = [calculator_tool, word_length_tool]

print("✅ Tools created:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")


### 9.2 Creating an Agent with Tools

Agents can use tools to perform actions and answer questions.


In [ ]:
if llm:
    # Create an agent using the ReAct framework
    # ReAct: Reasoning + Acting - the agent reasons about what to do, then acts
    
    from langchain.agents import AgentExecutor, create_react_agent
    from langchain import hub
    
    # Get the ReAct prompt template
    try:
        prompt = hub.pull("hwchase17/react")
    except:
        # Fallback prompt if hub is not available
        from langchain_core.prompts import PromptTemplate
        prompt = PromptTemplate.from_template("""
You are a helpful assistant that can use tools to answer questions.

You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")
    
    # Create the agent
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    # Test the agent
    print("Testing agent with calculation:")
    result = agent_executor.invoke({"input": "What is 15 * 8 + 23?"})
    print(f"\nFinal Answer: {result['output']}")
else:
    print("⚠️  LLM not available. Skipping agent example.")


### 9.3 Agent with Vector Store Tool

Agents can use vector stores as tools for retrieval.


In [ ]:
if vectorstore and llm:
    from langchain.tools.retriever import create_retriever_tool
    
    # Create a retriever tool
    retriever_tool = create_retriever_tool(
        vectorstore.as_retriever(),
        "knowledge_base",
        "Searches and returns information about machine learning, AI, and related topics."
    )
    
    # Combine tools
    all_tools = [retriever_tool, calculator_tool]
    
    # Create agent with retrieval capability
    try:
        prompt = hub.pull("hwchase17/react")
    except:
        from langchain_core.prompts import PromptTemplate
        prompt = PromptTemplate.from_template("""
You are a helpful assistant with access to a knowledge base and calculator.

Tools: {tools}
Tool Names: {tool_names}

Use the format:
Question: {input}
Thought: {agent_scratchpad}
""")
    
    agent = create_react_agent(llm, all_tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=all_tools, verbose=True)
    
    # Test with a question that requires retrieval
    print("Testing agent with knowledge retrieval:")
    result = agent_executor.invoke({"input": "What is machine learning? Also calculate 10 * 5."})
    print(f"\nFinal Answer: {result['output']}")
else:
    print("⚠️  Vector store or LLM not available. Skipping retrieval agent example.")


## 10. Memory

**Memory** allows chains and agents to remember information from previous interactions. This is essential for conversational applications.

### Types of Memory:

- **ConversationBufferMemory**: Stores all conversation history
- **ConversationBufferWindowMemory**: Stores only the last N messages
- **ConversationSummaryMemory**: Summarizes conversation history
- **ConversationSummaryBufferMemory**: Combines summary and buffer

### 10.1 Conversation Buffer Memory

Stores the entire conversation history.


In [ ]:
if llm:
    from langchain.chains import ConversationChain
    
    # Create memory
    memory = ConversationBufferMemory()
    
    # Create a conversation chain
    conversation = ConversationChain(
        llm=llm,
        memory=memory,
        verbose=True
    )
    
    # First interaction
    print("First interaction:")
    response1 = conversation.predict(input="Hi, my name is Alice. I love machine learning.")
    print(f"AI: {response1}\n")
    
    # Second interaction (should remember the name)
    print("Second interaction:")
    response2 = conversation.predict(input="What's my name and what do I love?")
    print(f"AI: {response2}\n")
    
    # Check memory
    print("Memory contents:")
    print(memory.buffer)
else:
    print("⚠️  LLM not available. Skipping memory example.")


### 10.2 Conversation Buffer Window Memory

Stores only the last N messages to limit memory usage.


In [ ]:
if llm:
    # Create window memory (keeps last 2 exchanges)
    window_memory = ConversationBufferWindowMemory(k=2)
    
    conversation = ConversationChain(
        llm=llm,
        memory=window_memory,
        verbose=False
    )
    
    # Multiple interactions
    conversation.predict(input="I'm learning Python programming.")
    conversation.predict(input="I also like data science.")
    conversation.predict(input="What programming language am I learning?")
    
    print("Window Memory (last 2 exchanges):")
    print(window_memory.buffer)
else:
    print("⚠️  LLM not available. Skipping window memory example.")


### 10.3 Memory with LCEL Chains

Using memory with modern LCEL chains.


In [ ]:
if llm:
    from langchain_core.chat_history import InMemoryChatMessageHistory
    from langchain_core.chat_messages import HumanMessage, AIMessage
    from langchain_core.runnables.history import RunnableWithMessageHistory
    
    # Create a simple chain
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Remember information from the conversation."),
        ("placeholder", "{messages}"),
        ("human", "{input}")
    ])
    
    chain = prompt | llm | output_parser
    
    # Create message history store
    store = {}
    
    def get_session_history(session_id: str):
        if session_id not in store:
            store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]
    
    # Add message history
    chain_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="messages"
    )
    
    # Use the chain with history
    config = {"configurable": {"session_id": "conversation_1"}}
    
    print("First message:")
    result1 = chain_with_history.invoke(
        {"input": "My favorite color is blue."},
        config=config
    )
    print(f"AI: {result1}\n")
    
    print("Second message (should remember):")
    result2 = chain_with_history.invoke(
        {"input": "What's my favorite color?"},
        config=config
    )
    print(f"AI: {result2}")
else:
    print("⚠️  LLM not available. Skipping LCEL memory example.")


## 11. Practical Examples

Let's build some practical applications combining multiple LangChain components.

### 11.1 Document Q&A System

A complete system that loads documents, creates embeddings, and answers questions.


In [ ]:
if llm and api_key:
    # Step 1: Load and split documents
    if 'documents' in locals() and documents:
        # Use existing documents
        doc_chunks = text_splitter.split_documents(documents)
    else:
        # Create sample documents
        sample_docs = [
            "LangChain is a framework for building LLM applications. It provides modular components.",
            "Chains allow you to combine multiple LLM calls and tools in sequence.",
            "Agents can autonomously use tools to complete tasks.",
            "Vector stores enable semantic search over documents using embeddings."
        ]
        doc_chunks = [{"page_content": doc} for doc in sample_docs]
    
    # Step 2: Create vector store
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    if isinstance(doc_chunks[0], dict):
        texts = [doc["page_content"] for doc in doc_chunks]
        vectorstore = Chroma.from_texts(texts, embeddings)
    else:
        vectorstore = Chroma.from_documents(doc_chunks, embeddings)
    
    # Step 3: Create RAG chain
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the question based on the context. If you don't know, say so.\n\nContext: {context}"),
        ("human", "{question}")
    ])
    
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    qa_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | output_parser
    )
    
    # Step 4: Ask questions
    questions = [
        "What is LangChain?",
        "What are chains used for?"
    ]
    
    for question in questions:
        print(f"Q: {question}")
        answer = qa_chain.invoke(question)
        print(f"A: {answer}\n")
else:
    print("⚠️  LLM or API key not available. Skipping Q&A system example.")


### 11.2 Conversational Agent with Tools

An agent that can have conversations and use tools.


In [ ]:
if llm:
    # Create tools
    tools = [calculator_tool, word_length_tool]
    
    # Create agent with memory
    try:
        prompt = hub.pull("hwchase17/react-chat")
    except:
        from langchain_core.prompts import MessagesPlaceholder
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant with access to tools."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
    
    agent = create_react_agent(llm, tools, prompt)
    
    # Add memory
    from langchain.agents import AgentExecutor
    from langchain.memory import ConversationBufferMemory
    
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True
    )
    
    # Test the conversational agent
    print("Conversation 1:")
    result1 = agent_executor.invoke({"input": "Hi, I'm Bob. Calculate 25 * 4 for me."})
    print(f"Response: {result1['output']}\n")
    
    print("Conversation 2 (should remember name):")
    result2 = agent_executor.invoke({"input": "What's my name and what was the calculation result?"})
    print(f"Response: {result2['output']}")
else:
    print("⚠️  LLM not available. Skipping conversational agent example.")


## 12. Best Practices and Tips

### 12.1 Error Handling

Always handle errors gracefully in production applications.


In [ ]:
if llm:
    # Example: Safe chain invocation with error handling
    def safe_invoke(chain, input_data):
        try:
            result = chain.invoke(input_data)
            return {"success": True, "result": result}
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    # Test with valid input
    chain = ChatPromptTemplate.from_messages([
        ("human", "{question}")
    ]) | llm | output_parser
    
    result = safe_invoke(chain, {"question": "What is AI?"})
    print("Result:", result)
else:
    print("⚠️  LLM not available. Skipping error handling example.")


### 12.2 Streaming Responses

Stream responses for better user experience.


In [ ]:
if llm:
    # Create a chain
    chain = ChatPromptTemplate.from_messages([
        ("human", "Write a short story about {topic}")
    ]) | llm
    
    # Stream the response
    print("Streaming response:")
    print("-" * 50)
    for chunk in chain.stream({"topic": "a robot learning to code"}):
        if hasattr(chunk, 'content'):
            print(chunk.content, end="", flush=True)
    print("\n" + "-" * 50)
else:
    print("⚠️  LLM not available. Skipping streaming example.")


### 12.3 Key Takeaways

1. **Use LCEL**: Prefer LCEL (`|` operator) for building chains - it's more flexible and modern
2. **Chunk Documents**: Always split large documents into smaller chunks for better retrieval
3. **Manage Memory**: Use appropriate memory types based on your use case (buffer vs window vs summary)
4. **Error Handling**: Always wrap LLM calls in try-except blocks
5. **Streaming**: Use streaming for better UX in production applications
6. **Tool Descriptions**: Write clear tool descriptions so agents know when to use them
7. **Temperature**: Adjust temperature based on task (lower for factual, higher for creative)
8. **Vector Stores**: Choose the right vector store for your scale (Chroma for small, Pinecone/Weaviate for large)

## 13. Next Steps

Now that you understand LangChain fundamentals, you can:

1. **Build RAG Applications**: Combine document loaders, vector stores, and chains
2. **Create Agents**: Build autonomous systems with tools and memory
3. **Explore Advanced Features**: 
   - LangGraph for complex agent workflows
   - LangSmith for observability and debugging
   - Custom tools and integrations
4. **Production Deployment**: 
   - Add error handling and retries
   - Implement caching
   - Set up monitoring and logging

## Resources

- **LangChain Documentation**: https://python.langchain.com/
- **LangChain Hub**: https://smith.langchain.com/hub
- **LangSmith**: https://smith.langchain.com/
- **GitHub**: https://github.com/langchain-ai/langchain

---

**Congratulations!** You've learned the fundamentals of the LangChain framework. Practice building your own applications and explore the advanced features!


In [ ]:
# Cleanup: Remove temporary files
import os
if os.path.exists("sample_ml.txt"):
    os.remove("sample_ml.txt")
    print("✅ Cleaned up temporary files")
